# TP Sytèmes de recommandation

Alban de Crevoisier

## 1 Présentation du modèle

In [75]:
import numpy as np
from numpy.linalg import norm
from scipy import sparse
from scipy.sparse.linalg import svds
from scipy.optimize import check_grad
from scipy.optimize import brent
from timeit import timeit

### Question 1.1

#### Que fait l'option `minidata` de la fonction `load_movielens` de `movielensutils.py` ?


In [5]:
def load_movielens(filename, minidata=False):
    """
    Cette fonction lit le fichier filename de la base de donnees
    Movielens, par exemple 
    filename = '~/datasets/ml-100k/u.data'
    Elle retourne 
    R : une matrice utilisateur-item contenant les scores
    mask : une matrice valant 1 si il y a un score et 0 sinon
    """

    data = np.loadtxt(filename, dtype=int)

    R = sparse.coo_matrix((data[:, 2], (data[:, 0]-1, data[:, 1]-1)),
                          dtype=float)
    R = R.toarray()  # not optimized for big data

    # code la fonction 1_K
    mask = sparse.coo_matrix((np.ones(data[:, 2].shape),
                              (data[:, 0]-1, data[:, 1]-1)), dtype=bool)
    mask = mask.toarray()  # not optimized for big data

    if minidata is True:
        R = R[0:100, 0:200].copy()
        mask = mask[0:100, 0:200].copy()

    return R, mask

L'option `minidata` réduit les données de travail aux notes des 100 premiers utilisateurs pour les 200 premiers films.

### Question 1.2

#### Combien y a-t-il d'utilisateurs, de films référencés dans la base de données ? Quel est le nombre total de notes ?

Il y a :
* 943 utilisateurs : nombre de lignes de R
* 1682 films : nombre de colonnes de R
* 100 000 notes : donné par `print(np.sum(m_mask))`

### Question 1.3

#### La fonction objectif est-elle convexe ?

Soit $f$ la fonction objectif :
$$
f: (P, Q) \mapsto \frac{1}{2}\|\mathbb{1}_K \circ (R - QP)\|_F^2 + \frac{\rho}{2}\|P\|_F^2 + \frac{\rho}{2}\|Q\|_F^2
$$
On remaque que $f$ est une forme quadratique.

Fixons $\rho = 0$ et $|U| = |C| = |I| = 1$. $f$ s'écrit alors : $f(p, q) = \frac{1}{2}(r - qp)^2$.

Soit $H(f)$ la matrice hessienne de f :
$$
H(f) =
\begin{bmatrix}
q^2 & 2qp -r \\
2qp -r & p^2
\end{bmatrix}
$$
Donc
$$
det(H(f)) = p^2q^2 - (2pq - r)^2 = (r - pq)(3pq - r)
$$

Pour $p = q = 0$ et $r = 1$, $det(H(f)) = -1$, la hessienne n'est donc pas définie positive, $f$ n'est donc pas convexe.

#### Quel est son gradient ?

$$
\nabla{f}(P,Q) =
\begin{bmatrix}
\nabla{f_Q} \\
\nabla{f_P}
\end{bmatrix}
= \begin{bmatrix}
-Q^T(\mathbb{1}_K \circ (R - QP)) + \rho P \\
(\mathbb{1}_K \circ (R-QP))(-P^T) + \rho Q
\end{bmatrix}
$$

#### Est-il lipschitzien ? Donner la constance de Lipschitz le cas échéant.

Si le gradient était Lipschitzien la hessienne serait bornée, or ici la hessienne est un polynôme d'ordre 2, $f$ n'est donc pas de gradient Lipschitzien.

## 2 Trouver $P$ quand $Q_0$ est fixé
$$
g: P \mapsto \frac{1}{2}\|\mathbb{1}_K \circ (R - Q^0P)\|_F^2 + \frac{\rho}{2}\|P\|_F^2 + \frac{\rho}{2}\|Q^0\|_F^2
$$

### Question 2.1

#### La fonction objectif $g$ est-elle convexe ?

# TODO

$P \mapsto \frac{\rho}{2}\|P\|_F^2$ est convexe.

#### Quel est son gradient ?

$$
\nabla{g}(P) = -(Q^0)^T(\mathbb{1}_K \circ (R-Q^0P)) + \rho P
$$

### Question 2.2

In [6]:
def objective(P, Q0, R, mask, rho):
    """
    La fonction objectif du probleme simplifie.
    Prend en entree 
    P : la variable matricielle de taille C x I
    Q0 : une matrice de taille U x C
    R : une matrice de taille U x I
    mask : une matrice 0-1 de taille U x I
    rho : un reel positif ou nul

    Sorties :
    val : la valeur de la fonction
    grad_P : le gradient par rapport a P
    """

    tmp = (R - Q0.dot(P)) * mask

    val = np.sum(tmp ** 2)/2. + rho/2. * (np.sum(Q0 ** 2) + np.sum(P ** 2))

    grad_P = -Q0.transpose().dot(tmp) + rho*P

    return val, grad_P

Vérification de l'implémentation :

In [9]:
R, mask = load_movielens('ml-100k/u.data', True)
Q0, s, P0 = svds(R, k=4)
rho = 0.3
check_grad(
    lambda p: objective(np.reshape(p, np.shape(P0)), Q0, R, mask, rho)[0],
    lambda p: np.ravel(objective(np.reshape(p, np.shape(P0)), Q0, R, mask, rho)[1]),
    np.ravel(P0))

0.0045040886682332819

L'erreur étant négligeable, l'implémentation est satisfaisante.

### Question 2.3

In [47]:
def gradient(g, P0, gamma, epsilon):
    """
    Minimise g par la méthode du gradient a pas constant.
    Prend en entree
    g : fonction a minimiser, Pk -> g(Pk), grad_g(Pk)
    P0 : point de depart
    gamma : pas constant
    epsilon : critere d'arret, norme de Frobenius du gradient de g en Pk
              inferieur ou egal a epsilon
    
    Sorties :
    Pk : minimiseur
    """
    
    grad_G = g(P0)[1]
    Pk = P0
    while np.sum(grad_G**2) > epsilon:
        Pk = Pk - gamma*grad_G
        grad_G = g(Pk)[1]
    
    return Pk

### Question 2.4

$\epsilon = 1$

In [48]:
epsilon = 1
L = rho + np.sum(Q0.transpose().dot(Q0)**2)
gradient(lambda P: objective(P, Q0, R, mask, rho), P0, 1/L, epsilon)

array([[ -1.65419321e+00,   3.44541590e+00,   1.94291022e+00,
         -6.45481399e-01,   9.95661542e-01,  -7.34057330e-02,
         -6.24716092e+00,  -3.04944178e+00,  -1.06806889e+00,
         -9.37619677e-01,  -4.50907466e+00,  -3.55533918e+00,
          9.27863159e-01,   9.26635810e-01,   3.73262118e+00,
          1.09894221e+00,  -5.48401336e-01,   1.33408284e+00,
         -8.54374160e-01,   1.45599603e+00,  -1.47111818e+00,
         -8.01556810e-01,  -5.87006814e+00,   1.95675267e-01,
          2.84785036e+00,   4.03307248e+00,   5.01878122e-03,
         -6.05804651e-01,   1.05752746e+00,   3.45918426e-01,
         -6.66950815e-01,  -4.26129337e+00,   1.60676199e+00,
          1.16960974e+00,   2.46071392e+00,   1.19125112e+00,
          1.26761105e+00,   4.77912697e+00,   2.18399353e+00,
          1.49403939e+00,   1.09870613e-01,  -1.57685441e+00,
          4.58453859e+00,   3.08324777e+00,   1.03654417e+00,
          2.77095264e+00,   1.90105542e+00,   2.21870669e+00,
        

## 3 Raffinements algorithmiques pour le problème à $Q_0$ fixé

### Question 3.1

In [82]:
def gradient_linear(g, P0, epsilon):
    """
    Minimise g par la methode du gradient avec recherche lineaire.
    Prend en entree
    g : fonction a minimiser, Pk -> g(Pk), grad_g(Pk)
    P0 : point de depart
    epsilon : critere d'arret, norme de Frobenius du gradient de g en Pk inferieur ou egal a epsilon

    Sorties :
    Pk : minimiseur
    """

    a, b, beta = 1/2, 1/2, 10**(-4) # Armijo's linear search parameters

    grad_G = g(P0)[1]
    Pk = P0
    while norm(grad_G) > epsilon:
        # Armijo's linear search
        l = 0
        while g(Pk-b*(a**l)*grad_G)[0] > g(Pk)[0] - beta*b*(a**l)*norm(grad_G):
            l = l + 1
        Pk = Pk - b*(a**l)*grad_G
        b = 2*b*(a**l)
        grad_G = g(Pk)[1]

    return Pk

In [83]:
gradient_linear(lambda P: objective(P, Q0, R, mask, rho), P0, epsilon)

array([[ -1.66706101e+00,   3.38656137e+00,   1.85089301e+00,
         -7.17891195e-01,   9.15295013e-01,  -1.87446397e-01,
         -6.24029676e+00,  -3.06742215e+00,  -1.08708493e+00,
         -1.14771650e+00,  -4.54279242e+00,  -3.56408382e+00,
          9.09884725e-01,   8.13840127e-01,   3.67720904e+00,
          1.05197651e+00,  -6.98577111e-01,   1.28260658e+00,
         -9.15126412e-01,   1.44702372e+00,  -1.55724396e+00,
         -8.25223660e-01,  -5.87124164e+00,   1.18495074e-01,
          2.80249453e+00,   4.01161826e+00,  -7.09281554e-02,
         -6.34127960e-01,   1.04991590e+00,   2.22599927e-01,
         -7.22736986e-01,  -4.27982186e+00,   1.54736141e+00,
          1.18045043e+00,   2.49611570e+00,   1.20995777e+00,
          1.27895770e+00,   4.75952107e+00,   2.10185420e+00,
          1.37747875e+00,   1.38176905e-02,  -1.56674101e+00,
          4.60108335e+00,   3.01957217e+00,   9.35461781e-01,
          2.79516595e+00,   1.80439669e+00,   2.07293793e+00,
        

### Question 3.2

Gradient conjugué ? g forme quadratique

In [55]:
def gradient_conjugate(g, P0, epsilon):
    """
    Minimise g par la methode du gradient conjugue : Fletcher et Reeves.
    Prend en entree
    g : fonction a minimiser, Pk -> g(Pk), grad_g(Pk)
    P0 : point de depart
    
    Sorties :
    Pk : minimiseur
    """
    
    grad_G = g(P0)[1]
    d = -grad_G
    Pk = P0
    while np.linalg.norm(grad_G) > epsilon:
        s = brent(lambda s: g(Pk + s*d)[0])
        Pk = Pk + s*d
        prev_grad = grad_G
        grad_G = g(Pk)[1]
        b = np.sum(grad_G**2) / np.sum(prev_grad**2)
        d = -grad_G + b*d

        return Pk

In [56]:
gradient_conjugate(lambda p: objective(p, Q0, R, mask, rho), P0, epsilon)

array([[  7.14354014e-01,   3.43111540e+00,   1.93896408e+00,
          1.46763953e+00,   1.16220964e+00,   9.37652685e-01,
         -7.18393590e+00,  -6.33027190e-01,   5.70797104e-01,
          7.72844021e-01,  -2.23623442e+00,  -2.04844227e+00,
          1.75380491e+00,   2.86271946e+00,   6.53347600e+00,
          8.21384211e-01,   5.29294180e-01,   1.28501677e+00,
          2.35054131e-01,   1.04234604e+00,  -4.04560397e-01,
          1.56426962e+00,  -3.16963496e+00,   1.21494016e+00,
          6.00318173e+00,   2.97691091e+00,   5.96020303e-01,
          1.89929323e+00,   9.33853422e-01,   1.09781461e+00,
          3.19961308e-01,  -1.98544895e+00,   2.03979393e+00,
          6.57511143e-01,   1.33511434e+00,   6.22903352e-01,
          7.29968188e-01,   4.08673023e+00,   2.58694699e+00,
          1.72102238e+00,   4.03409446e-01,  -9.57980820e-02,
          3.12258687e+00,   2.54746969e+00,   2.36633430e+00,
          1.54175434e+00,   3.02865749e+00,   4.04884048e+00,
        

### Question 3.3

Mesure des performances des algorithmes.

In [84]:
timeit(gradient(lambda P: objective(P, Q0, R, mask, rho), P0, 1/L, epsilon))
timeit(gradient_linear(lambda P: objective(P, Q0, R, mask, rho), P0, epsilon))
timeit(gradient_conjugate(lambda p: objective(p, Q0, R, mask, rho), P0, epsilon))

ValueError: stmt is neither a string nor callable

## 4 Résolution du problème complet

### Question 4.1

In [57]:
def total_objective(P, Q, R, mask, rho):
    """
    La fonction objectif du probleme complet.
    Prend en entree 
    P : la variable matricielle de taille C x I
    Q : la variable matricielle de taille U x C
    R : une matrice de taille U x I
    mask : une matrice 0-1 de taille U x I
    rho : un reel positif ou nul

    Sorties :
    val : la valeur de la fonction
    grad_P : le gradient par rapport a P
    grad_Q : le gradient par rapport a Q
    """

    tmp = (R - Q.dot(P)) * mask

    val = np.sum(tmp ** 2)/2. + rho/2. * (np.sum(Q ** 2) + np.sum(P ** 2))

    grad_P = rho * P - Q.T.dot(tmp)

    grad_Q = rho * Q - tmp.dot(P.T)

    return val, grad_P, grad_Q

In [58]:
 def rechercheLineaire(g,P0,Q0,epsilon):
    P=P0
    Q=Q0
    val, grad_P,grad_Q = total_objective(P=P, Q=Q, R=R , mask=mask, rho=0.3)
    norm_grad = np.sqrt(np.sum(grad_P**2)+np.sum(grad_Q**2))
    while (norm_grad > epsilon):
        l=0
        a,b,beta=(0.5,2.0,0.5)
        gamma=b
        Pprime = P - gamma*grad_P
        Qprime = Q - gamma*grad_Q
        valprime, grad_Pprime,grad_Qprime= total_objective(P=Pprime, Q=Qprime, R=R , mask=mask, rho=0.3)
        num=val+ beta*(np.trace(grad_P.T.dot(Pprime-P))+np.trace(grad_Q.T.dot(Qprime-Q)))
        while(valprime> num):
            l=l+1
            gamma=b*a**l
            Pprime = P - gamma*grad_P
            Qprime = Q - gamma*grad_Q
            valprime, grad_Pprime,grad_Qprime= g(P=Pprime, Q=Qprime, R=R , mask=mask, rho=0.3)
            num=val+ beta*(np.trace(grad_P.T.dot(Pprime-P))+np.trace(grad_Q.T.dot(Qprime-Q)))
        P=Pprime
        Q=Qprime
       
        grad_P=grad_Pprime
        grad_Q=grad_Qprime
        val=valprime
        norm_grad = np.sqrt(np.sum(grad_P**2)+np.sum(grad_Q**2))
    return val,P,Q

### Question 4.2

### Question 4.3

### Question 4.4

### Question 4.5